# CHÍNH THỨC

In [8]:
#pip install sqlalchemy

In [7]:
import os
from io import BytesIO
import pandas as pd
from minio import Minio
from minio_io_manager import load_data_from_MinIO, load_files_to_minio, raw_data_in_minio
from psql_io_manager import dataframe_to_postgres, query_to_dataframe

## config


In [12]:
minio_config = {
    "endpoint": "localhost:9000",
    "access_key": "minio",
    "secret_key": "minio123",
    "secure": False
}

client = Minio(
    minio_config["endpoint"],
    access_key=minio_config["access_key"],
    secret_key=minio_config["secret_key"],
    secure=minio_config["secure"]
)

# psql_config = {
#         "user": "postgres",
#         "password": "postgres",
#         "host": "localhost",
#         "port": "5434", 
#         "database": "project"
#     }
psql_config = {
        "user": "postgres",
        "password": "123456",
        "host": "localhost",
        "port": "5432", 
        "database": "Seminar_cleaned_data"
    }

# Hàm load file lên Minio

In [11]:
# Sử dụng hàm để tải lên tất cả các tệp tin trong thư mục
directory_path = "C:/Users/ADMIN/Downloads/project/seminar_data"
bucket_name = "datalake"
load_files_to_minio(client, directory_path, bucket_name)

MaxRetryError: HTTPConnectionPool(host='localhost', port=9000): Max retries exceeded with url: /datalake?location= (Caused by ProtocolError('Connection aborted.', BadStatusLine('ÿ\x00\x00\x00\x00\x00\x00\x00\x01\x7fh\x01st:9000\r\n')))

## Hàm tải file MinIO xuống

In [5]:
bucket_name="datalake"
rawdata=raw_data_in_minio(client, bucket_name)
print(rawdata)

['name_basics.tsv', 'title_basics.tsv', 'title_crew.tsv', 'title_episode.tsv', 'title_principals.tsv', 'title_ratings.tsv']


In [6]:
# name_basics.tsv
df=load_data_from_MinIO(client, rawdata[0],bucket_name)
df.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"soundtrack,actor,miscellaneous","tt0072308,tt0031983,tt0050419,tt0053137"
1,nm0000002,Lauren Bacall,1924,2014,"actress,soundtrack","tt0037382,tt0117057,tt0075213,tt0038355"
2,nm0000003,Brigitte Bardot,1934,\N,"actress,soundtrack,music_department","tt0049189,tt0054452,tt0057345,tt0056404"
3,nm0000004,John Belushi,1949,1982,"actor,soundtrack,writer","tt0072562,tt0077975,tt0078723,tt0080455"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0069467,tt0050986,tt0050976,tt0083922"


In [6]:
# title_basics.tsv
df1=load_data_from_MinIO(client, rawdata[1],bucket_name)
df1.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [8]:
# title_crew.tsv
df2=load_data_from_MinIO(client, rawdata[2],bucket_name)
df2.head()

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,\N
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


In [9]:
# title_episode.tsv
df3=load_data_from_MinIO(client, rawdata[3],bucket_name)
df3.head()

,tconst,parentTconst,seasonNumber,episodeNumber
0,tt0041951,tt0041038,1,9
1,tt0042816,tt0989125,1,17
2,tt0042889,tt0989125,\N,\N
3,tt0043426,tt0040051,3,42
4,tt0043631,tt0989125,2,16


In [7]:
# title_principals.tsv
df4=load_data_from_MinIO(client, rawdata[4],bucket_name)
df4.head()

,tconst,ordering,nconst,category,job,characters
0,tt0000001,1,nm1588970,self,\N,"[""Self""]"
1,tt0000001,2,nm0005690,director,\N,\N
2,tt0000001,3,nm0374658,cinematographer,director of photography,\N
3,tt0000002,1,nm0721526,director,\N,\N
4,tt0000002,2,nm1335271,composer,\N,\N


In [8]:
df5=load_data_from_MinIO(client, rawdata[5],bucket_name)
df5.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2003
1,tt0000002,5.8,269
2,tt0000003,6.5,1898
3,tt0000004,5.5,178
4,tt0000005,6.2,2682


# Cleaning

In [9]:
# replace double quotes in primaryTitle
df1["primaryTitle"] = df1["primaryTitle"].str.replace('"', '').astype(str)

# remove columns originalTitle, isAdult, startYear, endYear, runtimeMinutes
df1 = df1.drop(['originalTitle', 'isAdult', 'startYear', 'endYear', 'runtimeMinutes'], axis=1)

# remove values other than 'movie'
df1 = df1[df1['titleType'] == 'movie']

# remove genres == r'\N' or genres.isnull()
df1 = df1[(df1['genres'] != r'\N') & (df1['genres'].notnull())]

# remove titleType columns 
df1 = df1.drop(['titleType'], axis=1)

In [10]:
df1

,tconst,titleType,primaryTitle,genres
8,tt0000009,movie,Miss Jerry,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,"Documentary,News,Sport"
570,tt0000574,movie,The Story of the Kelly Gang,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,Drama
610,tt0000615,movie,Robbery Under Arms,Drama
...,...,...,...,...
10218010,tt9916622,movie,Rodolpho Teóphilo - O Legado de um Pioneiro,Documentary
10218037,tt9916680,movie,De la ilusión al desconcierto: cine colombiano...,Documentary
10218049,tt9916706,movie,Dankyavar Danka,Comedy
10218059,tt9916730,movie,6 Gunn,Drama


In [11]:
# remove columns: ordering, job, characters
df4 = df4.drop(['ordering', 'job', 'characters'], axis=1)

# remove category == 'self', 'cinematographer', 'production_designer', 'archive_footage', 'archive_sound'
df4 = df4[df4['category'] != 'self']
df4 = df4[df4['category'] != 'cinematographer']
df4 = df4[df4['category'] != 'production_designer']
df4 = df4[df4['category'] != 'archive_footage']
df4 = df4[df4['category'] != 'archive_sound']

In [12]:
df4

,tconst,nconst,category
1,tt0000001,nm0005690,director
3,tt0000002,nm0721526,director
4,tt0000002,nm1335271,composer
5,tt0000003,nm0721526,director
6,tt0000003,nm1770680,producer
...,...,...,...
58781934,tt9916880,nm0584014,director
58781935,tt9916880,nm0996406,director
58781936,tt9916880,nm1482639,writer
58781937,tt9916880,nm2586970,writer


# Insert các dataframe vào postgres (warehouse)

- __KHÔNG CHẠY__
- __CHỈ LOAD NHỮNG FILE TRAIN VÀO POSTGRES__

In [ ]:
dataframe_to_postgres(df1, psql_config, 'title_basics')

In [ ]:
dataframe_to_postgres(df4, psql_config, 'title_principals')

In [16]:
dataframe_to_postgres(df5, psql_config, 'title_ratings')